### Imports

In [75]:
import gradio as gr
import pandas as pd
import numpy as np
import os

In [87]:
def greet(name, num):
    return "Hello " + name + "!", num * 5

def square(num1, num2):
    return (num1 + num2) ** 2

with gr.Blocks() as demo:
    # First block
    name_input = gr.Textbox(label="Name")
    num_input = gr.Number(label="Number")
    greeting_output = gr.Textbox(label="Greeting")
    result_output = gr.Number(label="Result")
    greet_button = gr.Button("Greet")
    greet_button.click(greet, [name_input, num_input], [greeting_output, result_output])
    
    # Second block
    num_input2 = gr.Number(label="Number")
    square_output = gr.Number(label="Squared")
    square_button = gr.Button("Square")
    square_button.click(square, [result_output, num_input2], square_output)
    
    demo.launch()

Running on local URL:  http://127.0.0.1:7916

To create a public link, set `share=True` in `launch()`.


In [123]:
def import_csv(file):
    df = pd.read_csv(file.name)
    return df
        
def merge_dfs(df1, df2):
    result = pd.concat([df1, df2], axis=0)
    return result

with gr.Blocks() as demo:
    # First block
    file_input = gr.File()
    df_output = gr.Dataframe(type='pandas')
    submit = gr.Button("Submit")
    submit.click(import_csv, file_input, df_output)

    # Second block
    df_input1 = gr.Dataframe(interactive=True)
    df_input2 = gr.Dataframe(interactive=True)
    merged_output = gr.Dataframe()
    merge_button = gr.Button("Merge")
    merge_button.click(merge_dfs, [df_input1, df_input2], merged_output)
    
    demo.launch()

Running on local URL:  http://127.0.0.1:7938

To create a public link, set `share=True` in `launch()`.


In [94]:
def my_function(input):
    output1 = input + "aaaaaaaaa"
    output2 = input + "bbbbbbbbb"
    return (output1, output2)

with gr.Blocks() as demo:
    # First block
    input = gr.inputs.Textbox()
    output1 = gr.outputs.Textbox()
    output2 = gr.outputs.Textbox()
    submit = gr.Button("Submit")
    
    submit.click(my_function, input, [output1, output2])
    
    demo.launch()

C:\Users\gcmar\.conda\envs\elo_311\Lib\site-packages\gradio\inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
C:\Users\gcmar\.conda\envs\elo_311\Lib\site-packages\gradio\deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
C:\Users\gcmar\.conda\envs\elo_311\Lib\site-packages\gradio\deprecation.py:40: UserWarning: `numeric` parameter is deprecated, and it has no effect
  warnings.warn(value)
C:\Users\gcmar\.conda\envs\elo_311\Lib\site-packages\gradio\outputs.py:22: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(


Running on local URL:  http://127.0.0.1:7923

To create a public link, set `share=True` in `launch()`.
